In [0]:
import math
import numpy as np
import boto3
import os
import matplotlib.pyplot as plt
import pdal
import json
import io
from laspy import read
import pyarrow as pa
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, BooleanType, MapType
from pyspark.sql.functions import lit, col
from sedona.spark import *

In [0]:
os.environ['AWS_ACCESS_KEY_ID'] = dbutils.secrets.get(scope="aws_geospatial_s3", key="access_key")
os.environ['AWS_SECRET_ACCESS_KEY'] = dbutils.secrets.get(scope="aws_geospatial_s3", key="secret_key")
os.environ['AWS_DEFAULT_REGION'] = 'eu-west-2'      # Match your bucket region

In [0]:
bucket_name = "revodata-databricks-geospatial"
las_key = "geospatial-dataset/point-cloud/washington/las-laz/1816.las"

# Create an S3 client
s3 = boto3.client('s3')

# Stream file to memory
response = s3.get_object(Bucket=bucket_name, Key=las_key)
las_data = io.BytesIO(response['Body'].read())

# Read from buffer
las = read(las_data)
print(f"Loaded {len(las.x)} points")

In [0]:
plt.figure(figsize=(10, 6))
plt.scatter(las.x, las.y, c=las.z, s=0.1, alpha=0.5)
plt.colorbar(label="Elevation (Z)")
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Point Cloud Preview")
display(plt.show())

In [0]:
config = SedonaContext.builder() .\
    config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-shaded-3.3_2.12:1.7.1,'
           'org.datasyslab:geotools-wrapper:1.7.1-28.5'). \
    getOrCreate()

sedona = SedonaContext.create(config)

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS geospatial.pointcloud;

In [0]:
dataset_bucket_name = "revodata-databricks-geospatial"
dataset_input_dir="geospatial-dataset/point-cloud/washington/grid"
gpkg_file = "pc_grid.gpkg"
df_pc_grid = sedona.read.format("geopackage").option("tableName", "grid").load(f"s3://{dataset_bucket_name}/{dataset_input_dir}/{gpkg_file}").withColumnRenamed("geom", "geometry")

In [0]:
df_pc_grid.createOrReplaceTempView("pc_grid_vw")

df = spark.sql(f"""
SELECT g.fid, 
ST_X(g.geometry) AS x,
ST_Y(g.geometry) AS y,
g.geometry AS geometry
FROM pc_grid_vw g
""")

In [0]:
df = df.limit(1)
display(df)

In [0]:
radius = 100
tile_grid = ["geospatial-dataset/point-cloud/washington/1816.las"]

In [0]:
# function to get all points lying within range of the defined radius from the viewpoint
def getPoints(tile_grid, radius, view_height):
    # Viewpoint
    center = np.array([x, y])

    # Gather points
    arraysX, arraysY, arraysZ = [], [], [] # list of arrays of X,Y,Z coords
    arrayDistances = [] # Horizontal distances
    arrayClasses = [] # Classifications
    toBeAdded = []
    for tile in tile_grid:
        response = s3.get_object(Bucket=bucket_name, Key=tile)
        las_data = io.BytesIO(response['Body'].read())
        # read the .las file
        inFile = read(las_data)
        coords = np.vstack((inFile.x, inFile.y)).transpose()
        elevation = inFile.z
        distances = np.sqrt(np.sum((coords - center)**2, axis=1))
        keep_points = np.logical_and(np.logical_and(np.logical_or.reduce((
        inFile.classification == 5,
        inFile.classification == 6
        )),
        distances < radius),
        elevation >= view_height/1000)
        # Get coordinates
        arraysX.append(inFile.x[keep_points])
        arraysY.append(inFile.y[keep_points])
        arraysZ.append(inFile.z[keep_points])
        # Get distances
        arrayDistances.append(distances[keep_points])
        # Get classifications
        arrayClasses.append(inFile.classification[keep_points])

    # Concatenate all information
    X, Y, Z = arraysX[0], arraysY[0], arraysZ[0]
    distances = arrayDistances[0]
    classes = arrayClasses[0]
    for arrayX, arrayY in zip(arraysX[1:], arraysY[1:]):
        X = np.hstack([X, arrayX])
        Y = np.hstack([Y, arrayY])
    for arrayZ in arraysZ[1:]:
        Z = np.hstack([Z, arrayZ])
    for arDist in arrayDistances[1:]:
        distances = np.hstack([distances, arDist])
    for arClass in arrayClasses[1:]:
        classes = np.hstack([classes, arClass])
    return X, Y, Z, distances, classes 

In [0]:
def getheight(tile_grid, x, y):
    center = np.array([x, y])
    pointheight = 0
    points_number = 0

    bucket_name = "revodata-databricks-geospatial"

    # Stream file to memory
    for tile in tile_grid:
        # read the .las file
        response = s3.get_object(Bucket=bucket_name, Key=tile)
        las_data = io.BytesIO(response['Body'].read())
        file_input = read(las_data)
        # keep groundpoints satisfying ground_rules:
        # classification 2 for ground, inside las file
        # keep points within radius of 5 metres
        ground_rules = np.logical_and(
        file_input.classification == 2,np.sqrt(np.sum((np.vstack((file_input.x,
        file_input.y)).transpose() - center) ** 2, axis=1)) <= 1)
        build_rules = np.logical_and(
        file_input.classification == 6,
        np.sqrt(np.sum((np.vstack((file_input.x,
        file_input.y)).transpose() - center) ** 2, axis=1)) <= 1)

        ground_points = file_input.points[ground_rules]
        build_points = file_input.points[build_rules]
        print(build_points)
        # make array with heights of each point
        if len(ground_points) > len(build_points):
            ground_point_heights = np.array((ground_points.z)).transpose()
        else:
            ground_point_heights = np.array((build_points.z)).transpose()

        if len(ground_point_heights) > 0:
            pointheight += float(np.sum(ground_point_heights))
            points_number += len(ground_point_heights)

    # get mean value of points' heights
    if points_number > 0:
        height = pointheight / points_number
        return height
    else:
        return 0

In [0]:


def calculate_SVF(radius, dome):
    obstructedArea = 0
    treeObstruction = 0
    buildObstruction = 0
    for i in range(0, 180):
        for j in range(0, 90):
            if dome[i, j] != 0:
                v = 90 - (j + 1)
                R = math.cos(v * math.pi / 180) * radius
                r = math.cos((v + 1) * math.pi / 180) * radius
                # calculate area of each obstructed sector (circular sector area calculation)
                cell_area = (math.pi / 180.0) * (R ** 2 - r ** 2)
                obstructedArea += cell_area
                if dome[i, j] in [5]:
                    treeObstruction += cell_area
                elif dome[i, j] == 6:
                    buildObstruction += cell_area
    circleArea = math.pi * (radius ** 2)
    # SVF: proportion of open area to total area
    SVF = (circleArea - obstructedArea) / circleArea
    treeObstructionPercentage = treeObstruction / circleArea
    buildObstructionPercentage = buildObstruction / circleArea
    return SVF, treeObstructionPercentage, buildObstructionPercentage

In [0]:
def createDome(X, Y, Z, dists, classes, view_height):
    # Initialize dome
    # Indices = (Azimuth, Elevation)
    dome = np.zeros((180, 90), dtype=int)
    domeDists = np.zeros((180, 90), dtype=int)
    if len(X) > 0:
        # Azimuths
        dX, dY = X - x, Y - y
        azimuths = np.arctan2(dY, dX) * 180 / math.pi - 90
        azimuths[azimuths < 0] += 360
        # Elevations
        dZ = Z - view_height / 1000

        elevations = np.arctan2(dZ, dists) * 180 / math.pi
        # Shade sectors
        # Array with dome indices, distances & classifications
        data = np.stack((azimuths // 2, elevations // 1, dists, classes),
        axis=-1)
        # Sort according to indices & classifications
        sortData = data[np.lexsort([data[:, 2], data[:, 1], data[:, 0]])]
        # Spot where azimuth & elevation values change
        azimuth_change = sortData[:, 0][:-1] != sortData[:, 0][1:]
        elevation_change = sortData[:, 1][:-1] != sortData[:, 1][1:]
        keep = np.where(np.logical_or(azimuth_change, elevation_change))
        # Take position of next element, plus add first row
        shortestDistance = sortData[
        np.insert(keep[0] + 1, 0, 0)] # (inserts second element of change, first position, index of first point)
        # Define indices & classifications
        hor = shortestDistance[:, 0].astype(int)
        ver = shortestDistance[:, 1].astype(int)
        classif = shortestDistance[:, 3].astype(int)
        dists = shortestDistance[:, 2]
        # Update dome
        dome[hor, ver] = classif
        domeDists[hor, ver] = dists
        # Buildings as solids
        # Find building positions in dome
        # print dome[dome == 6].size
        if dome[dome == 6].size > 0:
            bhor, bver = np.where(dome == 6)
            # Create an array out of them
            builds = np.stack((bhor, bver), axis=-1)
            shape = (builds.shape[0] + 1, builds.shape[1])
            builds = np.append(builds, (bhor[0], bver[0])).reshape(shape)
            # Spot azimuth changes
            azimuth_change = builds[:, 0][:-1] != builds[:, 0][1:]
            keep = np.where(azimuth_change)
            # keep = np.insert(np.where(azimuth_change==True), 0, 0)
            # Change to building up to roof for each row
            roof_rows, roof_cols = builds[keep][:, 0], builds[keep][:, 1]
            for roof_row, roof_col in zip(roof_rows, roof_cols):
                condition = np.where(np.logical_or(domeDists[roof_row,:roof_col] > domeDists[roof_row, roof_col], dome[roof_row, :roof_col] == 0))
                dome[roof_row, :roof_col][condition] = 6
    plot(dome)
    return dome

In [0]:
# Plot dome
def plot(dome):
    # Create circular grid
    theta = np.linspace(0, 2*np.pi, 180, endpoint=False)
    radius = np.linspace(0, 90, 90)
    theta_grid, radius_grid = np.meshgrid(theta, radius)
    Z = dome.copy().astype(float)
    Z = Z.T[::-1, 0:]  # Transpose to (90, 180)
    # assign colors depending on class
    Z[Z == 0] = 0
    Z[np.isin(Z, [5])] = 0.5  # Classes 2-5
    Z[Z == 6] = 1
    if Z[Z == 6].size == 0:
        Z[0,0] = 1
        # Verify dimensions
    print(f"theta_grid: {theta.shape}, radius_grid: {radius.shape}, Z: {Z.shape}")
    axes = plt.subplot(111, projection='polar')
    cmap = plt.get_cmap('tab20c')
    axes.pcolormesh(theta, radius, Z, cmap=cmap)
    axes.set_ylim([0, 90])
    axes.tick_params(labelleft=False)
    axes.set_theta_zero_location("N")


In [0]:
tile_grid = ["geospatial-dataset/point-cloud/washington/las-laz/1816.las"]

x = 395176.7
y = 136693.5
radius = 100
view_height = getheight(tile_grid, x, y)
X, Y, Z, distances, classes = getPoints(tile_grid, radius, view_height)
print(view_height)

In [0]:
X, Y, Z, distances, classes = getPoints(tile_grid, radius, view_height)
print(classes)

In [0]:
dome = createDome(X, Y, Z, distances, classes, view_height)
plot(dome)

In [0]:
SVF, tree_percentage, build_percentage = calculate_SVF(radius, dome)
SVF, tree_percentage, build_percentage = round(SVF*100), round(tree_percentage*100), round(build_percentage*100)
print ('Sky: {}%'.format(int(SVF)) + "\n" +
'Vegetation {}%'.format(int(tree_percentage)) + "\n" +
'Building {}%'.format(int(build_percentage)))